In [6]:
import csv
import numpy as np
import pandas as pd


def tabbed_to_df(file_name: str):
    reader_output = []
    with open(file_name, newline='') as csvfile:
        f_reader = csv.reader(csvfile, delimiter='\t')
        for row in f_reader:
            reader_output.append(row)
    df = pd.DataFrame(data=reader_output,
                      columns=['Date', 'Price', 'Change', 'Change Percent', 'Change Cumulative Percent'],
                      dtype='float')
    df.set_index(['Date'], inplace=True)
    df.index = pd.to_datetime(df.index, format="%d/%m/%Y")
    return df


file_names = ['GAU', 'IPP']
file_paths = [f"{f}.txt" for f in file_names]
isbank_data = [tabbed_to_df(f) for f in file_paths]

all_data = pd.concat(isbank_data, keys=file_names)
all_data.drop(columns=['Change', 'Change Percent', 'Change Cumulative Percent'], inplace=True)

all_data = all_data.unstack(level=0).dropna().droplevel(0, axis=1)


In [40]:
first_day = all_data.resample('1W').apply(lambda x: x[0])
last_day = all_data.resample('1W').apply(lambda x: x[-1])
counts = all_data.resample('1W').count()

In [26]:
new_data = first_day.merge(last_day, on='Date', suffixes=('_first', '_last'))
new_data

,GAU_first,IPP_first,GAU_last,IPP_last
Date,,,,
2020-01-05,296.97,269.589430,298.93,269.660832
2020-01-12,305.91,269.868321,305.01,270.161147
2020-01-19,305.14,270.372690,304.34,270.645811
2020-01-26,306.73,270.850578,307.64,271.118667
2020-02-02,307.63,271.315683,312.74,271.582410
2020-02-09,313.83,271.783051,310.21,272.046494
2020-02-16,317.71,272.247827,316.62,272.503586
2020-02-23,317.21,272.698204,326.30,272.959084
2020-03-01,332.42,273.172818,338.27,273.436637


In [27]:
last_day.shift(1)

,GAU,IPP
Date,,
2020-01-05,NaN,NaN
2020-01-12,298.93,269.660832
2020-01-19,305.01,270.161147
2020-01-26,304.34,270.645811
2020-02-02,307.64,271.118667
2020-02-09,312.74,271.582410
2020-02-16,310.21,272.046494
2020-02-23,316.62,272.503586
2020-03-01,326.30,272.959084


In [43]:
haftalik_degisim = (first_day - last_day.shift(1))*100 / first_day
haftalik_degisim.sum()

GAU    13.023917
IPP     1.504494
dtype: float64

In [53]:
all_data.resample('1W').apply(lambda x: x[[-1]].index)

TypeError: cannot astype a datetimelike from [datetime64[ns]] to [float64]